# STM Transit Delay Data Preparation

## Data description

### Real-time Trip Updates

`current_time` timestamp when the data was collected<br>
`trip_id` unique identifier of a trip<br>
`route_id` bus line<br>
`start_date` start date of the trip<br>
`stop_id` stop number<br>
`arrival_time` actual arrival time, in milliseconds<br>
`departure_time` actual departure time, in milliseconds<br>
`schedule_relationship` state of the trip, 0 means scheduled and 1 means skipped

### Scheduled STM Trips

`trip_id` unique identifier of a trip<br>
`arrival_time` scheduled arrival time, in milliseconds<br>
`departure_time` scheduled departure time, in milliseconds<br>
`stop_id` stop number<br>
`stop_sequence` sequence of the stop, for ordering

### STM Stops

`stop_id` unique identifier of a stop<br>
`stop_code` bus stop or metro station number<br>
`stop_name` bus stop or metro station name<br>
`stop_lat` stop latitude<br>
`stop_lon` stop longitude<br>
`stop_url` stop web page<br>
`location_type` stop type<br>
`parent_station` parent station (metro station with multiple exits)<br>
`wheelchair_boarding` indicates if the stop is accessible for people in wheelchair, 1 being true and 2 being false

### Real-time Vehicle Positions

`current_time` timestamp when the data was collected<br>
`vehicle_id` unique identifuer of a vehicle<br>
`trip_id` unique identifier of a trip<br>
`route_id` bus or metro line<br>
`start_date` start date of a trip<br>
`start_time` start time of a trip<br>
`latitude` vehicle current latitude<br>
`longitude` vehicle current longityde<br>
`bearing` direction that the vehicle is facing<br>
`speed` momentary speed measured by the vehicle, in meters per second<br>
`stop_sequence` sequence of the stop, for ordering<br>
`status` vehicle stop status in relation with a stop that it's currently approaching or is at<br>
`timestamp` timestamp when STM updated the data<br>
`occupancy_status` degree of passenger occupancy

### Weather Archive and Forecast

`time` date and hour or the archived weather<br>
`temperature` air temperature at 2 meters above ground, in Celsius<br>
`precipitation` total precipitation (rain, showers, snow) sum of the preceding hour, in millimeters<br>
`windspeed` wind speed at 10 meters above ground, in km/h<br>
`weathercode` World Meteorological Organization (WMO) code

### Traffic Incidents

`category` category of the incident<br>
`start_time` start time of the incident in ISO8601 format<br>
`end_time` end time of the incident in ISO8601 format<br>
`length` length of the incident in meters<br>
`delay` delay in seconds caused by the incident (except road closures)<br>
`magnitude_of_delay` severity of the delay<br>
`last_report_time` date in ISO8601 format, when the last time the incident was reported<br>
`latitude` latitude of the incident<br>
`longitude` longitude of the incident

## Imports

In [1]:
from datetime import datetime, timedelta, UTC
from haversine import haversine, Unit
import pandas as pd
import sys

In [2]:
# Import custom code
sys.path.insert(0, '..')
from scripts.custom_functions import fetch_weather, LOCAL_TIMEZONE

In [3]:
trips_df = pd.read_csv('../data/fetched_stm_trip_updates.csv', low_memory=False)

In [4]:
schedules_df = pd.read_csv('../data/stop_times_2025-04-23.txt')

In [5]:
stops_df = pd.read_csv('../data/stops_2025-04-23.txt')

In [10]:
positions_df =  pd.read_csv('../data/fetched_stm_vehicle_positions.csv', low_memory=False)

In [6]:
weather_df = pd.read_csv('../data/fetched_historical_weather.csv')

In [7]:
traffic_df = pd.read_csv('../data/fetched_traffic.csv')

## Clean Data

In [8]:
# Convert route_id to integer
trips_df['route_id'] = trips_df['route_id'].str.extract(r'(\d+)')
trips_df['route_id'] = trips_df['route_id'].astype('int64')

In [9]:
# Sort trips
trips_df = trips_df.sort_values(by=['current_time', 'trip_id', 'route_id', 'arrival_time'])

In [11]:
# Get proportion of duplicates
subset = ['start_date', 'trip_id', 'route_id', 'stop_id']
duplicate_mask = trips_df.duplicated(subset=subset)
print(f'{(duplicate_mask.sum() / len(trips_df)):.2%}')

23.20%


In [12]:
# Remove duplicates
trips_df = trips_df.drop_duplicates(subset=subset, keep='last') # keep latest update

In [13]:
# Convert realtime arrival and departure time to milliseconds
trips_df['arrival_time'] = trips_df['arrival_time'] * 1000
trips_df['departure_time'] = trips_df['departure_time'] * 1000

In [14]:
# Get distribution of realtime arrival times
trips_df[['arrival_time', 'departure_time']].describe()

,arrival_time,departure_time
count,4.814260e+05,4.814260e+05
mean,1.645165e+12,1.640618e+12
std,4.069676e+11,4.154823e+11
min,0.000000e+00,0.000000e+00
25%,1.745809e+12,1.745809e+12
50%,1.745841e+12,1.745841e+12
75%,1.745856e+12,1.745856e+12
max,1.745875e+12,1.745875e+12


In [15]:
# Get proportion of rows with zero arrival times
zero_mask = trips_df['arrival_time'] == 0
print(f'{(zero_mask.sum() / len(trips_df)):.2%}')

5.77%


In [16]:
# Get proportion of rows where the arrival and departure times are different
diff_date_mask = trips_df['arrival_time'] != trips_df['departure_time']
print(f'{(diff_date_mask.sum() / len(trips_df)):.2%}')

5.90%


In [17]:
# Display rows
trips_df[diff_date_mask].sample(10)

,current_time,trip_id,route_id,start_date,stop_id,arrival_time,departure_time,schedule_relationship
260892,1.745838e+09,284738812,121,20250428,55934,1745840820000,0,0
498428,1.745856e+09,285283944,185,20250428,54039,0,1745856660000,0
295074,1.745838e+09,284739056,193,20250428,54571,0,1745838900000,0
251330,1.745834e+09,284738273,146,20250428,54093,1745836980000,0,0
512071,1.745860e+09,284777857,105,20250428,53936,1745861280000,0,0
89445,1.745802e+09,283553440,97,20250427,54010,1745804460000,0,0
231946,1.745834e+09,285009631,215,20250428,60383,0,1745837640000,0
179406,1.745816e+09,283854959,186,20250427,53725,0,1745817600000,0
268448,1.745838e+09,285002354,129,20250428,50898,1745839977000,0,0
392579,1.745845e+09,285010134,419,20250428,57822,0,1745846160000,0


In [18]:
# Replace zero arrival times by departure times, as they are usually the same
trips_df.loc[zero_mask, 'arrival_time'] = trips_df.loc[zero_mask, 'departure_time']

In [19]:
# Get proportion of rows with zero arrival times again
zero_mask = trips_df['arrival_time'] == 0
print(f'{(zero_mask.sum() / len(trips_df)):.2%}')

3.30%


In [20]:
# Delete the rows with 0 arrival times
trips_df = trips_df[~zero_mask]
zero_mask = trips_df['arrival_time'] == 0
assert zero_mask.sum() == 0

In [21]:
# Rename arrival time
trips_df = trips_df.rename(columns={'arrival_time': 'realtime_arrival_time'})

In [22]:
# Drop departure time
trips_df = trips_df.drop('departure_time', axis=1)

## Merge Data

### Realtime and Scheduled Trips

In [23]:
# Sort values by stop sequence
schedules_df = schedules_df.sort_values(by=['trip_id', 'stop_sequence'])

In [24]:
# Reset stop sequences (some stops might be missing)
schedules_df['stop_sequence'] = schedules_df.groupby('trip_id').cumcount() + 1

In [25]:
# Add trip progress
total_stops = schedules_df.groupby('trip_id')['stop_id'].transform('count')
schedules_df['trip_progress'] = schedules_df['stop_sequence'] / total_stops

In [26]:
# Merge realtime and scheduled trips
stm_trips_df = pd.merge(left=trips_df, right=schedules_df, how='inner', on=['trip_id', 'stop_id'])

In [27]:
# Convert start_date to datetime
stm_trips_df['start_date_dt'] = pd.to_datetime(stm_trips_df['start_date'], format='%Y%m%d')

In [28]:
def parse_gtfs_time(row) -> pd.Timestamp:
	'''
	Converts GTFS time string (e.g., '25:30:00') to datetime
	based on the arrival time.
	'''
	hours, minutes, seconds = map(int, row['arrival_time'].split(':'))
	total_seconds = hours * 3600 + minutes * 60 + seconds

	parsed_time = row['start_date_dt'] + timedelta(seconds=total_seconds)
	return parsed_time

In [29]:
# Convert planned arrival time to localized datetime
stm_trips_df['scheduled_arrival_time'] = stm_trips_df.apply(parse_gtfs_time, axis=1)
stm_trips_df['scheduled_arrival_time'] = stm_trips_df['scheduled_arrival_time'].dt.tz_localize(LOCAL_TIMEZONE)

In [30]:
# Convert planned time to timestamp in milliseconds since epoch
stm_trips_df['scheduled_arrival_time'] = stm_trips_df['scheduled_arrival_time'].astype('int64') // 10**6

### Trips and Stops

In [31]:
trips_stops_df = pd.merge(left=stm_trips_df, right=stops_df, how='inner', left_on='stop_id', right_on='stop_code')

In [32]:
trips_stops_df.columns

Index(['current_time', 'trip_id', 'route_id', 'start_date', 'stop_id_x',
       'realtime_arrival_time', 'schedule_relationship', 'arrival_time',
       'departure_time', 'stop_sequence', 'trip_progress', 'start_date_dt',
       'scheduled_arrival_time', 'stop_id_y', 'stop_code', 'stop_name',
       'stop_lat', 'stop_lon', 'stop_url', 'location_type', 'parent_station',
       'wheelchair_boarding'],
      dtype='object')

In [33]:
# Rename stop id
trips_stops_df = trips_stops_df.rename(columns={'stop_id_x': 'stop_id'})

In [34]:
# Convert wheelchair_boarding to boolean
trips_stops_df['wheelchair_boarding'] = (trips_stops_df['wheelchair_boarding'] == 1).astype('int64')

### Vehicle Positions

In [36]:
subset = ['trip_id', 'route_id', 'start_date', 'stop_sequence']
positions_df = positions_df.sort_values(by=subset)

In [37]:
duplicate_mask = positions_df.duplicated(subset=subset)
positions_df[duplicate_mask]

,current_time,vehicle_id,trip_id,route_id,start_date,start_time,latitude,longitude,bearing,speed,stop_sequence,status,timestamp,occupancy_status
15261,1.745852e+09,32802,914104,470,20250428,10:24:00,45.459759,-73.891930,144.0,0.00000,3,1,1745852444,1
15784,1.745853e+09,32802,914104,470,20250428,10:24:00,45.459759,-73.891930,94.0,0.00000,3,1,1745853345,1
16322,1.745854e+09,32802,914104,470,20250428,10:24:00,45.459759,-73.891930,0.0,0.00000,3,1,1745854247,1
16857,1.745855e+09,32802,914104,470,20250428,10:24:00,45.459759,-73.891930,115.0,0.00000,3,1,1745855147,1
17378,1.745856e+09,32802,914104,470,20250428,10:24:00,45.459759,-73.891930,0.0,0.00000,3,1,1745856049,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,1.745790e+09,28101,286594525,211,20250427,16:55:00,45.457355,-73.622627,46.0,23.05574,47,2,1745790294,3
6131,1.745820e+09,40082,286594572,195,20250428,00:48:00,45.507149,-73.772400,244.0,10.00008,53,2,1745820012,1
499,1.745790e+09,40042,286594779,114,20250427,18:03:00,45.424129,-73.610214,0.0,0.00000,1,1,1745790283,1
13704,1.745850e+09,40115,286598477,721,20250428,09:45:00,45.448807,-73.442589,0.0,0.00000,2,2,1745849756,1


In [38]:
# Get proportion of duplicates
print(f'{(duplicate_mask.sum() / len(positions_df)):.2%}')

2.14%


In [39]:
# Remove duplicates
positions_df = positions_df.drop_duplicates(subset=subset)

In [40]:
# Merge with other STM data
stm_df = pd.merge(left=trips_stops_df, right=positions_df, how='inner', on=['trip_id', 'route_id', 'start_date', 'stop_sequence'])

In [41]:
stm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22607 entries, 0 to 22606
Data columns (total 32 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   current_time_x          22607 non-null  float64       
 1   trip_id                 22607 non-null  int64         
 2   route_id                22607 non-null  int64         
 3   start_date              22607 non-null  int64         
 4   stop_id                 22607 non-null  int64         
 5   realtime_arrival_time   22607 non-null  int64         
 6   schedule_relationship   22607 non-null  int64         
 7   arrival_time            22607 non-null  object        
 8   departure_time          22607 non-null  object        
 9   stop_sequence           22607 non-null  int64         
 10  trip_progress           22607 non-null  float64       
 11  start_date_dt           22607 non-null  datetime64[ns]
 12  scheduled_arrival_time  22607 non-null  int64 

### STM and Weather

In [42]:
# Convert arrival timestamp to datetime
rt_arrival_dt = pd.to_datetime(stm_df['realtime_arrival_time'], origin='unix', unit='ms', utc=True)

In [43]:
# Round arrival time to the nearest hour
stm_df['rounded_arrival_dt'] = rt_arrival_dt.dt.round('h')

In [44]:
# Format time to match weather data
stm_df['time'] = stm_df['rounded_arrival_dt'].dt.strftime('%Y-%m-%dT%H:%M')

In [45]:
# Merge STM data with historical weather
stm_weather_df = pd.merge(left=stm_df, right=weather_df, how='left', on='time')

In [46]:
# Filter rows with null weather
null_weather_mask = stm_weather_df.isna().any(axis=1)

In [47]:
# Get proportion of rows with null weather
print(f'{(null_weather_mask.sum() / len(stm_weather_df)):.2%}')

100.00%


In [48]:
# Separate null and non null rows
not_null_df = stm_weather_df[~null_weather_mask]
null_df = stm_weather_df[null_weather_mask]

In [49]:
# Fetch forecast weather
start_date = null_df['rounded_arrival_dt'].min().strftime('%Y-%m-%d')
end_date = null_df['rounded_arrival_dt'].max().strftime('%Y-%m-%d')

weather_list = fetch_weather(start_date=start_date, end_date=end_date, forecast=True)
weather_df = pd.DataFrame(weather_list)

In [50]:
# Merge null weather dataframe with forecast
null_df = null_df.drop(['temperature', 'precipitation', 'windspeed', 'weathercode'], axis=1)
null_df = pd.merge(left=null_df, right=weather_df, how='inner', on='time')

In [51]:
# Merge null and non null weather dataframes
stm_weather_df = pd.concat([not_null_df, null_df]).reset_index()

### Traffic Data

In [52]:
# Get proportion of duplicates
duplicate_mask = traffic_df.duplicated()
print(f'{(duplicate_mask.sum() / len(traffic_df)):.2%}')

39.89%


In [53]:
# Remove duplicates
traffic_df = traffic_df.drop_duplicates(keep='last').reset_index()

In [54]:
# Convert STM arrival timestamp to datetime 
stm_weather_df['arrival_time_dt'] = pd.to_datetime(stm_weather_df['realtime_arrival_time'], origin='unix', unit='ms', utc=True)

In [55]:
# Convert traffic start_time and end_time to datetime
traffic_df['start_time_dt'] = pd.to_datetime(traffic_df['start_time'], utc=True)
traffic_df['end_time_dt'] = pd.to_datetime(traffic_df['end_time'], utc=True)

In [56]:
# Sort by date
traffic_df = traffic_df.sort_values(by='start_time_dt').reset_index()

In [57]:
# Fill null end times with current time (assuming the incident is still ongoing)
traffic_df['end_time_dt'] = traffic_df['end_time_dt'].fillna(datetime.now(UTC).replace(microsecond=0))
assert traffic_df['end_time_dt'].isna().sum() == 0

In [58]:
# Build traffic cache
def build_traffic_cache(traffic_df:pd.DataFrame) -> dict:
	traffic_cache = {}
	traffic_df['hour'] = traffic_df['start_time_dt'].dt.floor('h')

	for (hour, group) in traffic_df.groupby('hour'):
		traffic_cache[hour] = group.copy()

	return traffic_cache

Since there are many trip updates on the same day (even the same hour), there's a risk of repeating the filtering of active traffic incidents for each trip individually, which takes a lot of time for a large dataset. Traffic incidents are stable over minutes or hours. This is why the incidents are cached by hour.

In [59]:
def calculate_nearby_incidents(trip_update:pd.Series, traffic_cache:dict, max_distance:int=500) -> pd.Series:
	trip_datetime = trip_update['arrival_time_dt']
	stop_coords = (trip_update['stop_lat'], trip_update['stop_lon'])

	trip_hour = trip_datetime.floor('h')

	# Get cached incidents
	hour_incidents = traffic_cache.get(trip_hour)

	# Stop if there are no incidents for that hour
	if hour_incidents is None or hour_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})

	# Filter for active incidents at that trip hour
	active_incidents = hour_incidents[
		(hour_incidents['start_time_dt'] <= trip_datetime) &
		(hour_incidents['end_time_dt'] >= trip_datetime)
	].copy()

	if active_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})

	# Calculate distance     
	active_incidents['distance'] = active_incidents.apply(
		lambda row: haversine(stop_coords, (row['latitude'], row['longitude']), unit=Unit.METERS),
		axis=1
	)

	# Filter nearby
	nearby_incidents = active_incidents[active_incidents['distance'] <= max_distance]

	if nearby_incidents.empty:
		return pd.Series({
			'incident_nearby': 0,
			'nearest_incident_distance': None,
			'incident_category': None,
			'incident_delay': None,
			'incident_delay_magnitude': None
		})
	else:
		nearest = nearby_incidents.loc[nearby_incidents['distance'].idxmin()]
		return pd.Series({
			'incident_nearby': 1,
			'nearest_incident_distance': nearest['distance'],
			'incident_category': nearest['category'],
			'incident_delay': nearest['delay'],
			'incident_delay_magnitude': nearest['magnitude_of_delay']
		})

In [60]:
# Get traffic columns (get incidents within 500 meters)
traffic_cache = build_traffic_cache(traffic_df)
traffic_cols = stm_weather_df.apply(lambda row: calculate_nearby_incidents(row, traffic_cache), axis=1)

In [61]:
# Merge the traffic
df = pd.concat([stm_weather_df, traffic_cols], axis=1)

## Export Data

In [62]:
# Remove columns with constant values
nunique = df.nunique()
no_constant = nunique[nunique > 1]
no_constant_cols = no_constant.index
df = df[no_constant_cols]
df.columns

Index(['index', 'current_time_x', 'trip_id', 'route_id', 'start_date',
       'stop_id', 'realtime_arrival_time', 'arrival_time', 'departure_time',
       'stop_sequence', 'trip_progress', 'start_date_dt',
       'scheduled_arrival_time', 'stop_id_y', 'stop_code', 'stop_name',
       'stop_lat', 'stop_lon', 'stop_url', 'wheelchair_boarding',
       'current_time_y', 'vehicle_id', 'start_time', 'latitude', 'longitude',
       'bearing', 'speed', 'status', 'timestamp', 'occupancy_status',
       'rounded_arrival_dt', 'time', 'temperature', 'windspeed', 'weathercode',
       'arrival_time_dt', 'incident_nearby', 'nearest_incident_distance',
       'incident_category', 'incident_delay', 'incident_delay_magnitude'],
      dtype='object')

In [64]:
# Keep relevant columns #TODO: Add more incident features if needed
df = df[[
  	'trip_id',
  	'vehicle_id', 
	'occupancy_status',
  	'route_id',
  	'stop_id',
  	'stop_lat',
  	'stop_lon',
	'stop_sequence',
  	'trip_progress',
  	'wheelchair_boarding',
  	'realtime_arrival_time',
    'scheduled_arrival_time',
  	'temperature',
  	#'precipitation',
  	'windspeed', 
	'weathercode',
  	'incident_nearby', 
	#'incident_category',
	#'nearest_incident_distance',
	#'incident_delay',
	#'incident_delay_magnitude'
]]

In [65]:
# Assert all values are not null
assert df.isna().sum().sum() == 0

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22607 entries, 0 to 22606
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   trip_id                 22607 non-null  int64  
 1   vehicle_id              22607 non-null  int64  
 2   occupancy_status        22607 non-null  int64  
 3   route_id                22607 non-null  int64  
 4   stop_id                 22607 non-null  int64  
 5   stop_lat                22607 non-null  float64
 6   stop_lon                22607 non-null  float64
 7   stop_sequence           22607 non-null  int64  
 8   trip_progress           22607 non-null  float64
 9   wheelchair_boarding     22607 non-null  int64  
 10  realtime_arrival_time   22607 non-null  int64  
 11  scheduled_arrival_time  22607 non-null  int64  
 12  temperature             22607 non-null  float64
 13  windspeed               22607 non-null  float64
 14  weathercode             22607 non-null

In [67]:
# Export data to CSV
df.to_csv('../data/stm_weather_traffic_merged.csv', index=False)

## End